In [1]:
%matplotlib widget
#%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import aseg_gdf2
from shapely.geometry import Point, LineString, MultiPoint, Polygon
from shapely.wkt import loads
from scipy.io import loadmat
from scipy import spatial
import matplotlib.pyplot as plt
import rasterio
import netCDF4
import h5py
import gc
import os
import glob
from scipy.spatial.ckdtree import cKDTree
from hydrogeol_utils import SNMR_utils, spatial_functions, AEM_utils
from hydrogeol_utils import plotting_utils as plot_utils

In [3]:
def nearest_neighbours(points, coords, points_required = 1, max_distance = 250.):
    """

    :param points: array of points to find the nearest neighbour for
    :param coords: coordinates of points
    :param points_required: number of points to return
    :param max_distance: maximum search radius
    :return:
    """
    # Initialise tree instance
    kdtree = cKDTree(data=coords)

    # iterate throught the points and find the nearest neighbour
    distances, indices = kdtree.query(points, k=points_required,
                                      distance_upper_bound=max_distance)
    print(distances)
    # Mask out infitnite distances in indices to avoid confusion
    mask = ~np.isfinite(distances)

    distances[mask] = np.nan

    return distances, indices

# Find the nearest neighbours within the maximum distance

def xy_2_var(grid_dict, xy, var):
    """
    Function for finding a variable for gridded AEM sections
    given an input easting and northing
    @ param: grid_dict :dictionary for gridded line data
    @ param: xy: numpy array with easting and northing
    @ param: var: string with variable name
    returns
    float: distance along line
    """
    utm_coords = np.column_stack((grid_dict['easting'],
                                  grid_dict['northing']))

    d, i = spatial_functions.nearest_neighbours(xy,
                                                utm_coords,
                                                points_required=1,
                                                max_distance=100.)
    if np.isnan(d[0]):
        return None

    else:
        near_ind = i[0]
    


        return grid_dict[var][near_ind]


In [4]:
infile = r"C:\Users\PCUser\Desktop\AEM\LCI\DalyR_WB_MGA52.nc"

lci_dat = netCDF4.Dataset(infile)

lci_coords = np.column_stack((lci_dat['easting'][:],
                          lci_dat['northing'][:]))

# Initialise tree instance for nearest neighbour searches
kdtree = cKDTree(data=lci_coords)


In [5]:
# Since we will be wanting to plot the sections lets first grid them

# Create an instance of plots for gridding the data

plots = plot_utils.ConductivitySectionPlot(lci_dat)


# Define some key variables which we want to inteprolate

cond_vars = ['conductivity', 'data_residual', 'depth_of_investigation']

plots.conductivity_variables = cond_vars


# Define the resolution of th sections
xres, yres = 16., 4.


In [6]:
lines = [109703]

In [7]:
lines = [101501, 101502, 103502, 103501, 104501,
         105601, 105901, 105602, 105902, 106302,
         109302, 109401, 109204, 109502, 109701,
         109703, 110201, 109201]

In [8]:

# Now grid the lines and save in memory

hdf5_dir = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\lci\hdf5"

if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)

gridded_vars ={}

# Saved these out to save time

#gridded_vars = plots.grid_variables(xres = xres, yres =yres, lines=lines,
#                                    resampling_method = 'linear', save_hdf5 = True,
#                                   return_dict = True, hdf5_dir = hdf5_dir)

for line in lines:
#    
    infile = os.path.join(hdf5_dir, str(line) + '.hdf5')
    f = h5py.File(infile, 'r')
    gridded_vars[line] = plot_utils.extract_hdf5_data(f, cond_vars)
    f = None
    gc.collect()

In [9]:
gridded_vars.keys()

dict_keys([101501, 101502, 103502, 103501, 104501, 105601, 105901, 105602, 105902, 106302, 109302, 109401, 109204, 109502, 109701, 109703, 110201, 109201])

In [10]:
# Now we bring in the Oolloo Jinduckin contact to plot

#inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\raster\Daly_Aquifers\export\topOolloo_BOM.tif"
inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\raster\Daly_Aquifers\export\tJind_BOM.tif"
#inRaster = r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\raster\Ollo_Jind_contact_mAHD.tif"

contact_dataset = rasterio.open(inRaster)

contact_elev = contact_dataset.read(1)

contact_elev[contact_elev == contact_dataset.get_nodatavals()] = np.nan


In [11]:
# We want to find the top of the Jinduckin for every gridded lci section

for line in gridded_vars.keys():
    
    # Get the coordinates
    utm_coords = np.column_stack((gridded_vars[line]['easting'],
                                  gridded_vars[line]['northing']))
    
    
    # sample the raster at the coordinates
    
    elevGen = contact_dataset.sample(utm_coords)
    
    gridded_vars[line]['topJind_elev'] = np.nan*np.ones(shape = utm_coords.shape[0],
                                                 dtype = np.float)
    
    for i in range(utm_coords.shape[0]):
        
        jindElev =  next(elevGen)

        if not np.isclose(jindElev,contact_dataset.get_nodatavals()[0]):
            gridded_vars[line]['topJind_elev'][i] = jindElev


In [12]:
df_interp = pd.read_csv(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\Jinduckin_Oolloo_rj_interp.csv")

df_interp = df_interp[['line ', 'fiducial ', 'easting ', 'northing ', 'elevation ', 'OJ_mAHD']]

df_interp['source'] = 'rjmcmc_pmaps'

# Sample the distance along the line

df_interp['dist_along_line'] = np.nan

for index, row in df_interp.iterrows():

    # get the line and coordinates
    line = row['line ']
    
    xy = np.array([row[['easting ','northing ']].values])
    
    try:
        df_interp.loc[index,'dist_along_line'] = xy_2_var(gridded_vars[line],
                                                   xy,'grid_distances')
    except KeyError:
        pass

In [13]:
# Now we bring in the AEM grids

inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\03_LCI\03_Depth_Slices\Grids_doi_Masked\*.ers"

cond = {}

for file in glob.glob(inRaster):
    layer = int(file.split('Con')[1].split('_')[0])
    cond_dataset = rasterio.open(file)
    cond[layer] = cond_dataset.read(1)



In [14]:
# This function stores the top of the conductor in the dataframe on a click
def onclick(event):
    if event.xdata != None and event.ydata != None:
        lci_picks['dist_along_line'].append(event.xdata),
        lci_picks['OJ_mAHD'].append(event.ydata)
        lci_picks['line '].append(line)
        lci_picks['source'].append('lci_section')
    
def section_plot(outfile = None):
    fig = plt.figure(figsize = (12,4))

    # These are for interactive widget mode
    fig.canvas.layout.width = '6in'
    fig.canvas.layout.height= '2in'


    ax1 = fig.add_axes([0.05, 0.85, 0.8, 0.1])
    ax2 = fig.add_axes([0.05, 0.05, 0.8, 0.8])
    cbar_ax = fig.add_axes([0.88, 0.05, 0.01, 0.2])
    
    panel_kwargs = [{'title': '',
                      'color': 'black',
                      'ylabel': 'data \n residual',
                      'legend': False},
                     {'title': 'LCI conductivity',
                      'max_depth': 450.,
                      'shade_doi': True,
                      'colourbar': True,
                      'colourbar_label': 'Conductivity (S/m)',
                      'log_plot': True,
                      'vmin': 0.001,
                      'vmax': 2.,
                      'cmap': 'jet',
                      'ylabel': 'elevation \n (mAHD)'}]

    res1 = plot_utils.plot_single_line(ax1, gridded_vars[line],
                                       'data_residual', panel_kwargs[0])

    ax1.set_title('LCI conductivity section - ' + str(line))

    im2 = plot_utils.plot_grid(ax2, gridded_vars[line], 'conductivity',
                               panel_kwargs[1])
    
    # PLot jinduckin elevation onto section
    ax2.plot(gridded_vars[line]['grid_distances'],
            gridded_vars[line]['topJind_elev'], c = 'k',
            linestyle = 'dashed')
    
        
    cb2 = fig.colorbar(im2, cax=cbar_ax, orientation='vertical')
    
    cb2.ax.set_yticklabels([round(10 ** x, 4) for x in cb2.get_ticks()])
    cb2.set_label('conductivity (S/m)', fontsize=10)

    
    ax2.scatter(df_temp['dist_along_line'].values, df_temp['OJ_mAHD'].values,
               c = 'k', marker = '.')


    
    
    return fig, ax1, ax2

In [15]:
lines


[101501,
 101502,
 103502,
 103501,
 104501,
 105601,
 105901,
 105602,
 105902,
 106302,
 109302,
 109401,
 109204,
 109502,
 109701,
 109703,
 110201,
 109201]

In [25]:
plt.close('all')

#line = lines[18]
line = 109201
# subset the interpretations


df_temp = df_interp[df_interp['line '] == line]


fig, ax1, ax2 = section_plot()

min_d = 1
max_d = 50000



#ax1.set_xlim(min_d,max_d)
#ax2.set_xlim(min_d, max_d)

#plt.savefig(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\rjplots\\Florina_" + row['BORE_NO'] + ".png", dpi = 150)
cid=  fig.canvas.mpl_connect('button_press_event', onclick)

#cols = ['uniqueid ', 'survey ', 'date ', 'flight ', 'line ', 'fiducial ',
#       'easting ', 'northing ', 'elevation ', 'nchains ', 'nsamples ',
#       'nburnin ', 'sampletime ', 'misfit_lowest ', 'misfit_average ',
#        'topJind_interp']

#mask = gdf["topJind_interp"].notnull()
#
#gdf[cols].to_csv(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\Jinduckin_Oolloo_rj_interp_2.csv")

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:

df_interp = df_interp.append(pd.DataFrame(lci_picks), ignore_index = True)

#df_interp.reset_index(drop = True)

In [247]:
df_interp

,OJ_mAHD,dist_along_line,easting,elevation,fiducial,line,northing,source
0,-140.200203,NaN,742733.1,57.18,1126381.5,103101,8455589.0,rjmcmc_pmaps
1,-165.736171,NaN,744228.8,43.09,1128020.0,103301,8451584.0,rjmcmc_pmaps
2,-197.240365,NaN,744232.8,54.96,1139508.5,103801,8446599.0,rjmcmc_pmaps
3,-201.470042,NaN,745504.1,53.14,1141041.5,103901,8445593.0,rjmcmc_pmaps
4,-198.625849,NaN,746020.6,62.41,1139581.5,103801,8446597.0,rjmcmc_pmaps
...,...,...,...,...,...,...,...,...
358,146.214309,6428.872281,NaN,NaN,NaN,109703,NaN,lci_section
359,149.725548,7302.522281,NaN,NaN,NaN,109703,NaN,lci_section
360,160.259266,8003.172281,NaN,NaN,NaN,109703,NaN,lci_section
361,45.409200,63119.900000,NaN,NaN,NaN,110201,NaN,lci_section


In [246]:
plt.close()
plt.plot(df_interp.index)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [242]:
df_interp.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            ...
            60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
           dtype='int64', length=503)

In [249]:
df_interp.to_csv(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\Jinduckin_Oolloo_interp_lci_picks.csv", index = True)

In [144]:
#lci_picks = {'dist_along_line': [],
                     'OJ_mAHD': [], 'line ': [],
                     'source': []}

In [248]:
for index, row in df_interp.iterrows():
    line = row['line ']
    print(line)
    dist = row['dist_along_line']
    source = row['source']
    # TODO find nearest fiducial
    if source == 'lci_section':
        arr = gridded_vars[line]['grid_distances']
        idx = np.argmin(np.abs(arr - dist))
        easting = gridded_vars[line]['easting'][idx]
        northing = gridded_vars[line]['northing'][idx]
        
        df_interp.at[index, 'easting '] = easting
        df_interp.at[index, 'northing '] = northing
    

103101
103301
103801
103901
103801
103701
103701
103201
103201
103501
110102
110102
104001
103501
110102
110102
103501
110102
110102
103501
104401
103401
104401
110102
103501
110102
110102
110102
110102
105601
110102
110102
110102
105601
110102
110102
106301
105601
104601
104601
110103
106301
105601
110103
104601
106301
105601
106301
105601
105201
104901
106301
108001
105601
104601
105201
106301
108601
105601
105301
106301
105901
105601
106301
105601
106301
105601
105602
106301
105601
105602
109201
109201
106301
109201
105602
109201
109201
109201
109201
109201
105602
106301
109201
109201
109201
109201
105602
109201
109201
106301
109201
110301
109201
110301
110301
109201
110301
110301
105602
110301
110301
109201
110301
110203
110301
110301
110301
110301
106301
110301
110301
110301
110301
110301
110301
110301
109201
110301
105602
110203
110301
110301
106301
110203
105602
109201
110203
106301
105602
110203
106301
109201
105602
110203
110203
106301
109201
105602
110203
105602
106301
109201

In [228]:
df_interp

,OJ_mAHD,dist_along_line,easting,elevation,fiducial,line,northing,source
0,-140.200203,NaN,742733.1,57.18,1126381.5,103101,8455589.0,rjmcmc_pmaps
1,-165.736171,NaN,744228.8,43.09,1128020.0,103301,8451584.0,rjmcmc_pmaps
2,-197.240365,NaN,744232.8,54.96,1139508.5,103801,8446599.0,rjmcmc_pmaps
3,-201.470042,NaN,745504.1,53.14,1141041.5,103901,8445593.0,rjmcmc_pmaps
4,-198.625849,NaN,746020.6,62.41,1139581.5,103801,8446597.0,rjmcmc_pmaps
...,...,...,...,...,...,...,...,...
65,146.214309,6428.872281,NaN,NaN,NaN,109703,NaN,lci_section
66,149.725548,7302.522281,NaN,NaN,NaN,109703,NaN,lci_section
67,160.259266,8003.172281,NaN,NaN,NaN,109703,NaN,lci_section
68,45.409200,63119.900000,NaN,NaN,NaN,110201,NaN,lci_section


In [219]:
df_temp = df_interp[df_interp['line '] == line]

In [222]:
plt.close()

plt.plot(df_temp['easting '].values, df_temp['dist_along_line'].values)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Now we bring in the Oolloo Jinduckin contact to plot

inRaster = r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\raster\GA_LeapFrog_diff.tif"
diff_dataset = rasterio.open(inRaster)

diff_elev = diff_dataset.read(1)

diff_elev[diff_elev == diff_dataset.get_nodatavals()] = np.nan


In [29]:
fig, ax = plt.subplots(1, figsize = (6,4))
a = diff_elev.flatten()
a = a[~np.isnan(a)]
ax.hist(a, bins = 20)
ax.set_ylabel('bin count')
ax.set_xlabel('Jinduckin-Oolloo interface elevation difference (m)')
ax.axvline(np.median(a), ls = '--', c = 'k', label = 'median')
ax.legend()
plt.savefig(r'C:\Users\PCUser\Desktop\NSC_data\reporting\Daly\Figures\interface_difference_histogram.png',
           dpi = 200)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
a = diff_elev.flatten()
a = a[~np.isnan(a)]
print(np.median(a))
print(np.std(a))

-42.11932
68.399826
